# Multiword expressions identification and extraction

In [42]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import os
import math
import plotly.graph_objects as go
from IPython.display import HTML, display
import tabulate

In [2]:
nlp = Polish()

In [3]:
tokenizer = Tokenizer(nlp.vocab) # - w tej wersji interpunkcja nie jest oddzielana, ale poprawia szybkość działania znacząco

In [4]:
files_names = []
for file_name in os.listdir("../../ustawy"):
    if file_name.endswith(".txt"):
        files_names.append(os.path.join("../../ustawy", file_name))

In [5]:
bigram_dict = {}
tokens_freq = {}

In [6]:
for file_name in files_names:
    with open(file_name, encoding='utf-8') as file:
        bill = file.read()
        bill = bill.lower()
        tokens = tokenizer(bill)
        
        prev_token = tokens[0].text
        tokens_freq[prev_token] = tokens_freq.get(prev_token, 0) + 1
        for i in range(1,len(tokens)):
            token = tokens[i].text
            bigram = (prev_token, token)
            bigram_dict[bigram] = bigram_dict.get(bigram, 0) + 1
            prev_token = token
            tokens_freq[token] = tokens_freq.get(token, 0) + 1

In [7]:
len(bigram_dict)

1097691

In [8]:
key_to_delete = []
for key in bigram_dict.keys():
    if not key[0].isalpha() or not key[1].isalpha():
        key_to_delete.append(key)
        
for key in key_to_delete:
    bigram_dict.pop(key, None)

In [9]:
len(bigram_dict)

440612

In [10]:
bigram_dict = {k: v for k, v in sorted(bigram_dict.items(), key=lambda item: item[1], reverse=True)}
tokens_freq = {k: v for k, v in sorted(tokens_freq.items(), key=lambda item: item[1], reverse=True)}

## Pointwise mutual information

In [11]:
bigram_PMI = {}
tokens_amount = sum(tokens_freq.values())
bigrams_amount = sum(bigram_dict.values())

for bigram in bigram_dict.keys():
    word1, word2 = bigram
    prob_word1 = tokens_freq[word1] / tokens_amount
    prob_word2 = tokens_freq[word2] / tokens_amount
    prob_word1_word2 = bigram_dict[bigram] / bigrams_amount
    bigram_PMI[bigram] = math.log2(prob_word1_word2/(prob_word1*prob_word2)) 

In [12]:
bigram_PMI = {k: v for k, v in sorted(bigram_PMI.items(), key=lambda item: item[1], reverse=True)}

In [13]:
for bigram, pmi in list(bigram_PMI.items())[:10]:
    print(bigram[0] + ' ' + bigram[1] + ' ' + str(pmi))

doktorów habilitowanych 23.345369959864637
pionową ścianę 23.345369959864637
punktem wyprowadzonym 23.345369959864637
skrzynek lęgowych 23.345369959864637
usprawnianie zaburzonych 23.345369959864637
oświatowa nieobejmująca 23.345369959864637
stępkę położono 23.345369959864637
frachtem dystansowym 23.345369959864637
wybuchła wojna 23.345369959864637
dało pożytecznego 23.345369959864637


In [14]:
bigram_dict_leq5 = {k: v for k, v in bigram_dict.items() if v >= 5}

In [15]:
bigram_PMI_leq5 = {}
bigrams_leq5_amount = sum(bigram_dict_leq5.values())

for bigram in bigram_dict_leq5.keys():
    word1, word2 = bigram
    prob_word1 = tokens_freq[word1] / tokens_amount
    prob_word2 = tokens_freq[word2] / tokens_amount
    prob_word1_word2 = bigram_dict_leq5[bigram] / bigrams_leq5_amount
    bigram_PMI_leq5[bigram] = math.log2(prob_word1_word2/(prob_word1*prob_word2)) 

In [16]:
bigram_PMI_leq5 = {k: v for k, v in sorted(bigram_PMI_leq5.items(), key=lambda item: item[1], reverse=True)}

In [17]:
for bigram, pmi in list(bigram_PMI_leq5.items())[:10]:
    print(bigram[0] + ' ' + bigram[1] + ' ' + str(pmi))

ręcznego miotacza 21.480410711800463
świeckie przygotowujące 21.480410711800463
grzegorz schetyna 21.480410711800463
młyny kulowe 21.480410711800463
zaszkodzić wynikom 21.480410711800463
adama mickiewicza 21.21737630596667
przeponowe rurowe 21.21737630596667
mleczka makowego 21.21737630596667
schedę spadkową 20.994983884630223
lambrekiny okienne 20.994983884630223


## Log likelihood ratio 

In [18]:
bigram_occurs = sum(bigram_dict.values())
bigram_LLR = {}

In [19]:
def H(args, N):
    res = 0
    for k in args:
        k /= N
        k_eq_0 = 1 if k == 0 else 0
        res += k * math.log(k + k_eq_0)
    return res

In [20]:
for bigram, AandB in bigram_dict.items():
    word1, word2 = bigram
    BnoA = tokens_freq[word2] - AandB
    AnoB = tokens_freq[word1] - AandB
    noAnoB = bigram_occurs - (AandB + BnoA + AnoB)
    LLR = 2 * bigram_occurs * (H([AandB, BnoA, AnoB, noAnoB], bigram_occurs) - \
                               H([AandB + BnoA, AnoB + noAnoB], bigram_occurs) - \
                               H([AandB + AnoB, BnoA + noAnoB], bigram_occurs))
    bigram_LLR[bigram] = LLR     

In [21]:
bigram_LLR = {k: v for k, v in sorted(bigram_LLR.items(), key=lambda item: item[1], reverse=True)}

In [22]:
for bigram, llr in list(bigram_LLR.items())[:10]:
    print(bigram[0] + ' ' + bigram[1] + ' ' + str(llr))

mowa w 124645.83146718914
których mowa 97656.02403137869
o których 68833.82784639853
którym mowa 64283.83923316678
dodaje się 59756.50172767568
do spraw 50379.07917977016
o którym 46855.915324589136
w w 42404.08935560332
stosuje się 40152.69589803207
minister właściwy 39356.628900809366


## Trigram analysis 

In [23]:
trigram_dict = {}

In [24]:
for file_name in files_names:
    with open(file_name, encoding='utf-8') as file:
        bill = file.read()
        bill = bill.lower()
        tokens = tokenizer(bill)
        
        prev_token = tokens[0].text
        prev_token2 = tokens[1].text
        for i in range(2,len(tokens)):
            token = tokens[i].text
            trigram = (prev_token, prev_token2, token)
            trigram_dict[trigram] = trigram_dict.get(trigram, 0) + 1
            prev_token = prev_token2
            prev_token2 = token

In [25]:
len(trigram_dict)

2354226

In [26]:
key_to_delete_trigrams = []
for key in trigram_dict.keys():
    if not key[0].isalpha() or not key[1].isalpha() or not key[2].isalpha():
        key_to_delete_trigrams.append(key)

for key in key_to_delete_trigrams:
    trigram_dict.pop(key, None)

In [27]:
len(trigram_dict)

724349

In [28]:
trigram_dict = {k: v for k, v in sorted(trigram_dict.items(), key=lambda item: item[1], reverse=True)}

### Trigram PMI

In [29]:
trigram_PMI = {}
tokens_amount = sum(tokens_freq.values())
trigrams_amount = sum(trigram_dict.values())
    
for trigram in trigram_dict.keys():
    word1, word2, word3 = trigram
    prob_word1 = tokens_freq[word1] / tokens_amount
    prob_word2 = tokens_freq[word2] / tokens_amount
    prob_word3 = tokens_freq[word3] / tokens_amount

    prob_word1_word2_word3 = trigram_dict[trigram] / trigrams_amount
    trigram_PMI[trigram] = math.log(prob_word1_word2_word3 / (prob_word1 * prob_word2 * prob_word3))

In [30]:
trigram_PMI = {k: v for k, v in sorted(trigram_PMI.items(), key=lambda item: item[1], reverse=True)}

In [31]:
for trigram, pmi in list(trigram_PMI.items())[:10]:
    print(trigram[0] + ' ' + trigram[1] + ' ' + trigram[2] + ' ' + str(pmi))

fabryki portland cementu 31.872699325377354
wniebowzięcia najświętszej maryi 31.872699325377354
najświętszej maryi panny 31.872699325377354
suthma eegcou eok 31.872699325377354
przekazowi pieniężnemu towarzyszyły 31.872699325377354
wirusowych gorączek krwotocznych 31.872699325377354
nagminnemu porażeniu dziecięcemu 31.872699325377354
chleb świętojański strąkowy 31.872699325377354
implantacji stymulatora nerwu 31.872699325377354
prosimy uważnie przeczytać 31.872699325377354


In [32]:
trigram_dict_leq5 = {k: v for k, v in trigram_dict.items() if v >= 5}

In [33]:
trigram_PMI_leq5 = {}
trigrams_amount_leq5 = sum(trigram_dict_leq5.values())
    
for trigram in trigram_dict_leq5.keys():
    word1, word2, word3 = trigram
    prob_word1 = tokens_freq[word1] / tokens_amount
    prob_word2 = tokens_freq[word2] / tokens_amount
    prob_word3 = tokens_freq[word3] / tokens_amount

    prob_word1_word2_word3 = trigram_dict_leq5[trigram] / trigrams_amount_leq5
    trigram_PMI_leq5[trigram] = math.log(prob_word1_word2_word3 / (prob_word1 * prob_word2 * prob_word3))

In [34]:
trigram_PMI_leq5 = {k: v for k, v in sorted(trigram_PMI_leq5.items(), key=lambda item: item[1], reverse=True)}

In [35]:
for trigram, pmi in list(trigram_PMI_leq5.items())[:10]:
    print(trigram[0] + ' ' + trigram[1] + ' ' + trigram[2] + ' ' + str(pmi))

piłce nożnej uefa 27.55491435712318
profilem zaufanym epuap 27.53655710160481
religijne uroczystości pogrzebowe 27.495971821489732
finałowego turnieju mistrzostw 27.477622682821533
potwierdzonym profilem zaufanym 27.219022066003348
turnieju mistrzostw europy 27.215258418354043
grożącą jemu samemu 26.612955878500447
bankowemu funduszowi gwarancyjnemu 26.54591422714476
komunalne osady ściekowe 26.515142568478005
konfesyjne nauczanie religii 26.492669712625947


### Trigram LLR

In [36]:
trigram_LLR = {}

Traktujemy trigramy jak bigramy tak, że (word1,word2,word3) -> ((word1, word2),word3), czyli (bigram, unigram) zamiast (unigram,unigram,unigram).

In [37]:
trigram_occurs = sum(trigram_dict.values())

for trigram, AandBandC in trigram_dict.items():
    word1, word2, word3 = trigram
    
    CnoAandB = tokens_freq[word3] - AandBandC
    AandBnoC = bigram_dict[(word1, word2)] - AandBandC
    noAandBnoC = trigram_occurs - (AandBandC + CnoAandB + AandBnoC)
    LLR = 2 * trigram_occurs * (H([AandBandC, CnoAandB, AandBnoC, noAandBnoC], trigram_occurs) - \
                                H([AandBandC + CnoAandB, AandBnoC + noAandBnoC], trigram_occurs) - \
                                H([AandBandC + AandBnoC, CnoAandB + noAandBnoC], trigram_occurs))
    trigram_LLR[trigram] = LLR  

In [38]:
trigram_LLR = {k: v for k, v in sorted(trigram_LLR.items(), key=lambda item: item[1], reverse=True)}

In [39]:
for trigram, llr in list(trigram_LLR.items())[:10]:
    print(trigram[0] + ' ' + trigram[1] + ' ' + trigram[2] + ' ' + str(llr))

o których mowa 92100.04972834075
o którym mowa 62646.7332030165
których mowa w 47739.06848574327
właściwy do spraw 44653.074728556625
o której mowa 35850.49113124134
którym mowa w 32449.770015804836
ustawie z dnia 31176.668053809528
zastępuje się wyrazami 25215.325587729756
minister właściwy do 24519.717879464166
wejścia w życie 23589.36245948675


## Result comparison table 

In [56]:
bigram_table = [val for val in zip(list([k1 + ' ' + k2 for (k1,k2) in bigram_PMI.keys()])[:10], 
                                   list(bigram_PMI.values())[:10], 
                                   list([k1 + ' ' + k2 for (k1,k2) in bigram_PMI_leq5.keys()])[:10], 
                                   list(bigram_PMI_leq5.values())[:10],
                                   list([k1 + ' ' + k2 for (k1,k2) in bigram_LLR.keys()])[:10], 
                                   list(bigram_LLR.values())[:10])]

display(HTML('<h1>Bigrams table</h1>'))
display(HTML(tabulate.tabulate(bigram_table, tablefmt='html', headers=['PMI', 'PMI value', 'PMI>=5', 'PMI>=5 value',
                                                                 'LLR', 'LLR value'])))

PMI,PMI value,PMI>=5,PMI>=5 value,LLR,LLR value
doktorów habilitowanych,23.3454,ręcznego miotacza,21.4804,mowa w,124646
pionową ścianę,23.3454,świeckie przygotowujące,21.4804,których mowa,97656
punktem wyprowadzonym,23.3454,grzegorz schetyna,21.4804,o których,68833.8
skrzynek lęgowych,23.3454,młyny kulowe,21.4804,którym mowa,64283.8
usprawnianie zaburzonych,23.3454,zaszkodzić wynikom,21.4804,dodaje się,59756.5
oświatowa nieobejmująca,23.3454,adama mickiewicza,21.2174,do spraw,50379.1
stępkę położono,23.3454,przeponowe rurowe,21.2174,o którym,46855.9
frachtem dystansowym,23.3454,mleczka makowego,21.2174,w w,42404.1
wybuchła wojna,23.3454,schedę spadkową,20.995,stosuje się,40152.7
dało pożytecznego,23.3454,lambrekiny okienne,20.995,minister właściwy,39356.6


Można zauważyć, że żadne kolokacje nie powtarzają się w obrębie PMI. Jest to spowodowane silną kolokacją wyrazów występujących rzadko, ponieważ zapewne występują tylko raz i tym samym tylko razem. Natomiast po wykrojeniu tych bigramów, które są zbyt rzadko w zbiorze otrzymujemy wyrazy będące w rzeczywistości w silnej kolokacji, ponieważ występują w dużych ilościach i kilkukrotnie razem, ale jak widać nie zawsze, bo miara jest mniejsza niż w poprzedniej kolumnie. W przypadku LLR miara ta jest jeszcze bardziej restrykcyjna, ponieważ znajduje wyrazy, które rzeczywiście najczęściej występują razem spośród wszystkich kombinacji danych słów w bigramie. 

In [58]:
trigram_table = [val for val in zip(list([k1 + ' ' + k2 + ' ' + k3 for (k1,k2,k3) in trigram_PMI.keys()])[:10], 
                                   list(trigram_PMI.values())[:10], 
                                   list([k1 + ' ' + k2 + ' ' + k3 for (k1,k2,k3) in trigram_PMI_leq5.keys()])[:10], 
                                   list(trigram_PMI_leq5.values())[:10],
                                   list([k1 + ' ' + k2 + ' ' + k3 for (k1,k2,k3) in trigram_LLR.keys()])[:10], 
                                   list(trigram_LLR.values())[:10])]

display(HTML('<h1>Trigrams table</h1>'))
display(HTML(tabulate.tabulate(trigram_table, tablefmt='html', headers=['PMI', 'PMI value', 'PMI>=5', 'PMI>=5 value',
                                                                 'LLR', 'LLR value'])))

PMI,PMI value,PMI>=5,PMI>=5 value,LLR,LLR value
fabryki portland cementu,31.8727,piłce nożnej uefa,27.5549,o których mowa,92100
wniebowzięcia najświętszej maryi,31.8727,profilem zaufanym epuap,27.5366,o którym mowa,62646.7
najświętszej maryi panny,31.8727,religijne uroczystości pogrzebowe,27.496,których mowa w,47739.1
suthma eegcou eok,31.8727,finałowego turnieju mistrzostw,27.4776,właściwy do spraw,44653.1
przekazowi pieniężnemu towarzyszyły,31.8727,potwierdzonym profilem zaufanym,27.219,o której mowa,35850.5
wirusowych gorączek krwotocznych,31.8727,turnieju mistrzostw europy,27.2153,którym mowa w,32449.8
nagminnemu porażeniu dziecięcemu,31.8727,grożącą jemu samemu,26.613,ustawie z dnia,31176.7
chleb świętojański strąkowy,31.8727,bankowemu funduszowi gwarancyjnemu,26.5459,zastępuje się wyrazami,25215.3
implantacji stymulatora nerwu,31.8727,komunalne osady ściekowe,26.5151,minister właściwy do,24519.7
prosimy uważnie przeczytać,31.8727,konfesyjne nauczanie religii,26.4927,wejścia w życie,23589.4


W przypadku trigramów w PMI otrzymano znacznie więcej ciekawych kolokacji, które są np. nazwą instytucji jak np. bank funduszu gwarancyjnego. Tak jak w poprzednim przypadku bigramy w PMI>=5 są dużo bardziej sensowne niż PMI oraz LLR. Dla LLR silne kolokacje stworzyły wyrażenia łącznikowe, ale nie jesteśmy już w stanie na ich podstawie stwierdzić, co było np. tematem ustawy, a przy PMI już jak najbardziej.

Trigramy dają rzeczywistą informację o sile kolokacji w stosunku do bigramów. Częściej są to wyrażenia, które są tym, co chcielibyśmy uzyskać np. Turniej Mistrzostw Europy itp., czyli wyrażenia niosące zupełnie inne znaczenie niż każde ze słów je tworzących.

1) Why do we have to filter the bigrams, rather than the token sequence?

Ponieważ filtrowanie tokenów przez znalezieniem bigramów spowodowałoby utworzenie nieistniejących kolokacji. Na przykład zdanie: 'W ustawie z dn. 14 grudnia...' po usunięciu znaków niealfanumerycznych przed stworzeniem bigramów wyprodukowałoby bigram (dn, grudnia), który tak naprawdę nie istnieje.

2) Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?

Dla bigramów odpowiednią metodą wydaje się być PMI>=5, ale LLR także zwracało sensowne wyniki takie jak np. minister właściwy, dodaje się itp. Są to kolokacje na podstawie których można uzyskać jakieś dodatkowe informacje o tekście. Natomiast dla trigramów najlepszą metodą jest zdecydowanie PMI>=5 oraz PMI, ponieważ znajduje całe wyrażenia własne np. Wniebowzięcie Najświętszej Maryi, potwierdzony profil zaufany. Są to dane wprowadzające dużą wiedzą dodatkową na temat tekstu. Jednak wszystko zależy od tego co chcieliśmy uzyskać. Metoda LLR jest bardziej niezależna od tego, ile słów wchodzi w skład n-gramu, zawsze znajduje podobne kolokacje np. będące rozszerzeniem bigramu do trigramu.

3)What types of expressions are discovered by the methods.

PMI/PMI>=5 - imiona i nazwiska, nazwy świąt, wydarzeń sportowych, nazwy własne

LLR - silne kolokacje słów będących łącznikami zdań np. w którym, o których itp.

4) Can you devise a different type of filtering that would yield better results?

Można dokonać lepszej filtracji słów np. wyrzucić słowa krótsze niż 2 litery, wyrzucić tzw. stop words, pozbyć się słów najczęściej występujących równocześnie w wielu n-gramach, co wyeliminuje słowa typu których, które itp.